In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
root = '/gdrive/MyDrive/heartd/'
hdf = pd.read_csv(root + 'heart_cleveland_upload.csv')

In [ ]:
hdf

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,40,1,3,152,223,0,0,181,0,0.0,0,0,2,1
293,39,1,3,118,219,0,0,140,0,1.2,1,0,2,1
294,35,1,3,120,198,0,0,130,1,1.6,1,0,2,1
295,35,0,3,138,183,0,0,182,0,1.4,0,0,0,0


In [ ]:
from scipy import stats
hdf = hdf[(np.abs(stats.zscore(hdf)) < 3).all(axis=1)]
hdf = hdf.reset_index()

In [ ]:
def bayes(traindf,trgt,selectedcols,testdf):
    dfq = pd.DataFrame()
    target = pd.get_dummies(trgt)
    for cols in target:
      locals()[f'selecteddf{cols}'] = traindf.loc[trgt == cols,selectedcols]
      locals()[f'ffresults{cols}'] = []
    for cols in target:
      for x in range(0,len(testdf)):
        results1 = []
        for cols2 in selectedcols:
          res = ((1/np.sqrt(2*np.pi*np.var(eval(f'selecteddf{cols}')[cols2])))*np.exp(-(((testdf[cols2].iloc[x])-
                      np.average(eval(f'selecteddf{cols}')[cols2]))**2)/2*np.var(eval(f'selecteddf{cols}')[cols2])))
        res = res*(len(eval(f'selecteddf{cols}'))/len(traindf))
        fresults = np.prod(res)
        eval(f'ffresults{cols}').append(fresults)
    for cols in trgt:
      dfq[cols] = pd.Series(eval(f'ffresults{cols}'))
    dfq['evidence'] = dfq.sum(axis=1)
    dfq = dfq.div(dfq.evidence, axis=0)
    dfq = dfq.drop(['evidence'], axis=1)
    finalres = dfq.idxmax(axis=1)  
    return finalres

In [ ]:
from sklearn.model_selection import train_test_split
selectedcols = ['chol','trestbps','thalach']
trgt = hdf['condition']
df1 = hdf.drop(['condition'],axis=1)
df1=(df1 - np.average(df1)) / df1.std()
trainX, testX, trainy, testy = train_test_split(df1, trgt, test_size=0.2, random_state=40)
from sklearn.metrics import classification_report
print(classification_report(testy, bayes(trainX,trainy,selectedcols,testX)))

              precision    recall  f1-score   support

           0       0.72      0.94      0.82        31
           1       0.89      0.59      0.71        27

    accuracy                           0.78        58
   macro avg       0.81      0.76      0.76        58
weighted avg       0.80      0.78      0.77        58



In [ ]:
from sklearn.naive_bayes import GaussianNB
X = hdf[['chol','trestbps','thalach']]
y = hdf['condition']
X=(X - np.average(X)) / X.std()
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=40)
clf = GaussianNB()
clf.fit(trainX, trainy)
print(classification_report(testy,clf.predict(testX)))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79        31
           1       0.78      0.67      0.72        27

    accuracy                           0.76        58
   macro avg       0.76      0.75      0.75        58
weighted avg       0.76      0.76      0.76        58

